# 55 Public, Cleveland Simulations
8760 rows per simulation<br>
Floors for residential and commercial

## Preprocess

1. Generate zone with facade glazing systems in COMFEN.

2. Retrieve IDFs for each scenario from the COMFEN5 Results folder. Have to do each of the 3 sections of the building as its own sqlite file and each project by orientation separately. Then IDF files are copied after each simulation. These files will be used in the Jupyter Notebook.

3. Convert from E+ 8.1 to 9.5 using the .lst format.
4. Change the reporting in the IDF files to "All" to get the HTML summary table and the inch-pound units for verification.
      
    * OutputControl:Table:Style,
    * All,                     !- Column Separator
    * InchPound;               !- Unit Conversion
5. Rerun batch sims in EnergyPlus v9.5.
6. Add "Scenario", "Elevation, and "GlazingSystem" to the results CSV files.

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

## Energy Costs and Emission Factors

http://seeatcalc.gastechnology.org/ResidentialBuildings.aspx

In [2]:
# Costs (elec $/kWh, gas $/therm), change values per city
resgas_cost = 1.16
reselec_cost = 0.1238

comgas_cost = 0.56
comelec_cost = 0.0972

In [3]:
# Source Energy Conversion Factors (same for res or comm), change values per city
gas_source = 1.09
elec_source = 2.93

In [4]:
# Carbon equivalent factors (same for res or comm), change values per city

gas_co2 = 0.14566
elec_co2 = 0.42103

In [5]:
# Zone size for EUI

sf = 329.55

In [6]:
# Lighting energy reduction, assumption that LEDs are being used. 
# Use this multiplier if reductions are not in the COMFEN model

lighting_redux = 1

In [7]:
# # For peak demand calculations (.0002931 kW = 1 Btu).

# kw = 0.0002931

---

## Combine CSV Files in Directory Commercial Electric Heating

In [8]:
# Create variable for files in directory
files = [f for f in os.listdir("data/cle/ComElecElec/") if f.endswith(".csv")]

# files

In [9]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_ComElec = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/cle/ComElecElec/" + file)
    all_ComElec = pd.concat([all_ComElec, df])
    
all_ComElec

# 105120  rows, 108 columns

,Scenario,Elevation,GlazingSystem,Date/Time,Environment:Site Outdoor Air Drybulb Temperature [C](Hourly),COMFENZONE PEOPLE 1:People Occupant Count [](Hourly),COMFENZONE:Zone Lights Electricity Energy [J](Hourly),COMFENZONE:Zone Windows Total Heat Gain Rate [W](Hourly),COMFENZONE:Zone Windows Total Heat Gain Energy [J](Hourly),EXTWALL:Surface Outside Face Sunlit Fraction [](Hourly),...,COMFENZONE PEOPLE 1:Zone Thermal Comfort Fanger Model PPD [%](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 90% Acceptability Status [](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 80% Acceptability Status [](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Running Average Outdoor Air Temperature [C](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Temperature [C](Hourly),COMFENZONE:Zone Cooling Setpoint Not Met Time [hr](Hourly),SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly),SYS-1:Air System NaturalGas Energy [J](Hourly),SYS-1:Air System Fan Electricity Energy [J](Hourly),SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly)
0,cle_com_elec_10,E,Exist_LowE,01/01 01:00:00,-1.2000,0.0,11869.19907,0.0,0.0,0.0,...,77.977719,-1.0,-1.0,-0.886806,-1.0,0.0,1.533456e+06,0,6.662803e+05,0.0
1,cle_com_elec_10,E,Exist_LowE,01/01 02:00:00,-0.6000,0.0,11869.19907,0.0,0.0,0.0,...,74.840990,-1.0,-1.0,-0.886806,-1.0,0.0,1.787081e+06,0,8.883738e+05,0.0
2,cle_com_elec_10,E,Exist_LowE,01/01 03:00:00,-0.6000,0.0,11869.19907,0.0,0.0,0.0,...,77.609970,-1.0,-1.0,-0.886806,-1.0,0.0,1.437250e+06,0,6.662803e+05,0.0
3,cle_com_elec_10,E,Exist_LowE,01/01 04:00:00,-0.6000,0.0,11869.19907,0.0,0.0,0.0,...,74.922504,-1.0,-1.0,-0.886806,-1.0,0.0,1.860285e+06,0,8.883738e+05,0.0
4,cle_com_elec_10,E,Exist_LowE,01/01 05:00:00,-0.6000,0.0,11869.19907,0.0,0.0,0.0,...,77.471419,-1.0,-1.0,-0.886806,-1.0,0.0,1.584762e+06,0,6.662803e+05,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,cle_com_elec_09,N,Exist_LowE,12/31 20:00:00,3.4875,0.0,11869.19907,0.0,0.0,0.0,...,84.124350,-1.0,-1.0,-0.714479,-1.0,0.0,0.000000e+00,0,0.000000e+00,0.0
8756,cle_com_elec_09,N,Exist_LowE,12/31 21:00:00,3.5250,0.0,11869.19907,0.0,0.0,0.0,...,76.344404,-1.0,-1.0,-0.714479,-1.0,0.0,1.276364e+06,0,1.406848e+06,0.0
8757,cle_com_elec_09,N,Exist_LowE,12/31 22:00:00,3.6750,0.0,11869.19907,0.0,0.0,0.0,...,74.403663,-1.0,-1.0,-0.714479,-1.0,0.0,5.762689e+05,0,1.406848e+06,0.0
8758,cle_com_elec_09,N,Exist_LowE,12/31 23:00:00,4.2125,0.0,11869.19907,0.0,0.0,0.0,...,79.769622,-1.0,-1.0,-0.714479,-1.0,0.0,7.915104e+05,0,7.034239e+05,0.0


In [10]:
#list(all_ComElec.columns.values)

In [11]:
# Get wanted columns
all_ComElec1 = all_ComElec[["Scenario",
               "Date/Time",
                "Elevation",
               "GlazingSystem",
              "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)",
              "SYS-1:Air System NaturalGas Energy [J](Hourly)",
              "SYS-1:Air System Fan Electricity Energy [J](Hourly)",
              "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ",
             "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)"]]

#all_ComElec1

In [12]:
all_ComElec1 = all_ComElec1.dropna().reset_index(drop=True)

#all_ComElec1

In [13]:
# # Export grouped file to csv
# all_ComElec1.to_csv("Scraper_Output/cle_com_all.csv", header=True, index=True)

-----

## Get the Wanted Data (columns)

In [14]:
# Rename Columns
all_ComElec2 = all_ComElec1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "SYS-1:Air System Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "SYS-1:Air System NaturalGas Energy [J](Hourly)":"HeatingGasEnergy[J](Hourly)",
                            "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)":"HeatingElecEnergy[J](Hourly)",
                            "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly)",
                            "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)":"LightingEnergy[J](Hourly)"
                             })

#all_ComElec2

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>

In [15]:
# Copy dataframe to modify and leave original df intact
get_annual_ComElec = all_ComElec2.copy(deep=True)

#get_annual_ComElec

In [16]:
# Convert Joules to KBtu in dataframe
get_annual_ComElec["FanEnergy(kBtu)"] = get_annual_ComElec["FanEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["HeatingElecEnergy(kBtu)"] = get_annual_ComElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["HeatingGasEnergy(kBtu)"] = get_annual_ComElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["CoolingEnergy(kBtu)"] = get_annual_ComElec["CoolingEnergy[J](Hourly)"] * .00000094781712

# Reduce lighting energy, assuming LEDs are being used
get_annual_ComElec["LightingEnergy(kBtu)"] = (get_annual_ComElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

#get_annual_ComElec

In [17]:
# Drop columns
get_annual_ComElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_ComElec

In [18]:
# Add energy totals
totalComEnergy = get_annual_ComElec["FanEnergy(kBtu)"] + get_annual_ComElec["HeatingElecEnergy(kBtu)"] + get_annual_ComElec["LightingEnergy(kBtu)"] + get_annual_ComElec["CoolingEnergy(kBtu)"]
        
get_annual_ComElec["AnnualEnergy(kBtu)"] = totalComEnergy

# get_annual_ComElec

-----

## Get Energy per sf 
Add EUI (site) column<br>


In [19]:
# Get data per sf (energy/sf)
get_annual_ComElec["FanEnergy(kBtu/sf)"] = (get_annual_ComElec["FanEnergy(kBtu)"]) / sf
get_annual_ComElec["HeatingElecEnergy(kBtu/sf)"] = (get_annual_ComElec["HeatingElecEnergy(kBtu)"]) / sf
get_annual_ComElec["HeatingGasEnergy(kBtu/sf)"] = (get_annual_ComElec["HeatingGasEnergy(kBtu)"]) / sf
get_annual_ComElec["CoolingEnergy(kBtu/sf)"] = (get_annual_ComElec["CoolingEnergy(kBtu)"]) / sf
get_annual_ComElec["LightingEnergy(kBtu/sf)"] = (get_annual_ComElec["LightingEnergy(kBtu)"]) / sf

In [20]:
# Establish EUI (energy/sf)
get_annual_ComElec["EUI(kBtu/sf)"] = (get_annual_ComElec["FanEnergy(kBtu)"] +
                                    get_annual_ComElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ComElec["LightingEnergy(kBtu)"] +  
                                    get_annual_ComElec["CoolingEnergy(kBtu)"]) / sf
                                                       
#get_annual_ComElec

-----

## Get Carbon per sf 
Add carbon/sf column

In [21]:
# Do carbon calculation using variable (lb/sf)
get_annual_ComElec["CO2_Electric(lb/sf)"] = ((get_annual_ComElec["FanEnergy(kBtu)"] +
                                    get_annual_ComElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ComElec["LightingEnergy(kBtu)"] +
                                    get_annual_ComElec["CoolingEnergy(kBtu)"]) * elec_co2) / sf
                                                       
# get_annual_ComElec

In [22]:
get_annual_ComElec["CO2_Gas(lb/sf)"] = ((get_annual_ComElec["HeatingGasEnergy(kBtu)"]) * gas_co2) / sf
                                                       
# get_annual_ComElec

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Add cost multiplier, variables defined above

In [23]:
# Get annual costs
get_annual_costs_ComElec = all_ComElec2.copy(deep=True)

In [24]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs_ComElec["FanCost($)"] = (get_annual_costs_ComElec["FanEnergy[J](Hourly)"] * .000000277777778) * comelec_cost
get_annual_costs_ComElec["HeatingElecCost($)"] = ((get_annual_costs_ComElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * comgas_cost
get_annual_costs_ComElec["HeatingGasCost($)"] = ((get_annual_costs_ComElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * comgas_cost
get_annual_costs_ComElec["CoolingCost($)"] = (get_annual_costs_ComElec["CoolingEnergy[J](Hourly)"] * .000000277777778) * comelec_cost

# Reduce lighting energy, assuming LEDs are being used
get_annual_costs_ComElec["LightingCost($)"] = (get_annual_costs_ComElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

#get_annual_costs

In [25]:
# Drop columns
get_annual_costs_ComElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_costs_ComElec

In [26]:
# Add cost totals
totalComCost = get_annual_costs_ComElec["FanCost($)"] + get_annual_costs_ComElec["HeatingElecCost($)"] + get_annual_costs_ComElec["HeatingGasCost($)"]  + get_annual_costs_ComElec["CoolingCost($)"] + get_annual_costs_ComElec["LightingCost($)"]
        
get_annual_costs_ComElec["AnnualCost($)"] = totalComCost

# get_annual_costs_ComElec

-----

## Get Annual Cost per sf Data
Add costs/sf columns<br>

In [27]:
# Get data per sf ($/sf)
get_annual_costs_ComElec["FanCost($/sf)"] = (get_annual_costs_ComElec["FanCost($)"]) / sf
get_annual_costs_ComElec["HeatingElecCost($/sf)"] = (get_annual_costs_ComElec["HeatingElecCost($)"]) / sf
get_annual_costs_ComElec["HeatingGasCost($/sf)"] = (get_annual_costs_ComElec["HeatingGasCost($)"]) / sf
get_annual_costs_ComElec["CoolingCost($/sf)"] = (get_annual_costs_ComElec["CoolingCost($)"]) / sf
get_annual_costs_ComElec["LightingCost($/sf)"] = (get_annual_costs_ComElec["LightingCost($)"]) / sf

# get_annual_costs_ComElec

In [28]:
# Establish costs ($/sf)
get_annual_costs_ComElec["AnnualCost($/sf)"] = (get_annual_costs_ComElec["FanCost($)"] +
                                    get_annual_costs_ComElec["HeatingElecCost($)"] +
                                    get_annual_costs_ComElec["HeatingGasCost($)"] +  
                                    get_annual_costs_ComElec["CoolingCost($)"] +  
                                    get_annual_costs_ComElec["LightingCost($)"]) / sf
                                                       
# get_annual_costs_ComElec

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [29]:
# Combine data for energy and cost
allAnnualDataComElec = pd.merge(get_annual_ComElec, get_annual_costs_ComElec)

#allAnnualDataComElec

In [30]:
# list(allAnnualDataComElec.columns.values)

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Export to file

In [31]:
# Group by scenario
annualDataCom = allAnnualDataComElec.groupby(["Scenario", "Elevation", "GlazingSystem"], as_index=True)

# annualDataCom.sum().round(2)

In [32]:
annualDataComFinal = annualDataCom.sum().round(2)

# annualDataComFinal

In [33]:
# Export grouped file to csv
annualDataComFinal.to_csv("Scraper_Output/cle_annual_com.csv", header=True, index=True)

---

## Establish Annual Baseline at Existing

Create dataset with just baselines.<br> 
Keep the units in kBtus to be able to eventually combine heating and cooling <br>
Merge the baseline dataset with the full dataset.<br>
Determine the delta between scenario and base.

In [34]:
# Define the scenario for the baselines.
# % improvement from Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees)
baselineAnnualDataComElec = allAnnualDataComElec[allAnnualDataComElec["GlazingSystem"] == "Exist"]

# baselineAnnualDataComElec

In [35]:
# Rename columns to indicate baseline to prepare for merge"

allBaselineAnnualDataComElec = baselineAnnualDataComElec.rename(columns={"Scenario":"B_Scenario",
 'Date_Time':'B_Date_Time',
 'Elevation':'B_Elevation',
 'GlazingSystem':'B_GlazingSystem',
 'FanEnergy(kBtu)':'B_FanEnergy(kBtu)',
 'HeatingElecEnergy(kBtu)':'B_HeatingElecEnergy(kBtu)',
 'HeatingGasEnergy(kBtu)':'B_HeatingGasEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'B_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'B_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'B_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'B_FanEnergy(kBtu/sf)',
 'HeatingElecEnergy(kBtu/sf)':'B_HeatingElecEnergy(kBtu/sf)',
 'HeatingGasEnergy(kBtu/sf)':'B_HeatingGasEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'B_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'B_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'B_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'B_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'B_CO2_Gas(lb/sf)',
 'FanCost($)':'B_FanCost($)',
 'HeatingElecCost($)':'B_HeatingElecCost($)',
 'HeatingGasCost($)':'B_HeatingGasCost($)',
 'CoolingCost($)':'B_CoolingCost($)',
 'LightingCost($)':'B_LightingCost($)',
 'AnnualCost($)':'B_AnnualCost($)',
 'FanCost($/sf)':'B_FanCost($/sf)',
 'HeatingElecCost($/sf)':'B_HeatingElecCost($/sf)',
 'HeatingGasCost($/sf)':'B_HeatingGasCost($/sf)',
 'CoolingCost($/sf)':'B_CoolingCost($/sf)',
 'LightingCost($/sf)':'B_LightingCost($/sf)',
 'AnnualCost($/sf)':'B_AnnualCost($/sf)'
                            })

#allBaselineAnnualDataComElec

---
## Group and Export Baseline Annual Data
Group dataframe by scenario<br>
Export to file

In [36]:
# Group by scenario
annualBaselineDataCom = allBaselineAnnualDataComElec.groupby(["B_Scenario", "B_Elevation", "B_GlazingSystem"], as_index=True)

# annualDataCom.sum().round(2)

In [37]:
annualBaselineDataCom = annualBaselineDataCom.sum().round(2)

# annualDataComFinal

In [38]:
# list(AnnualWithBaselineCom.columns.values)

In [39]:
# Export grouped file to csv
annualBaselineDataCom.to_csv("Scraper_Output/cle_baseline_annual_com.csv", header=True, index=True)

## Find Peak Demand
Slice data to find peak demand for each scenario<br>
Export to csv

In [40]:
# Copy dataframe to modify and leave original df intact
get_peak_ComElec = all_ComElec2.copy(deep=True)

get_peak_ComElec

,Scenario,Date_Time,Elevation,GlazingSystem,HeatingElecEnergy[J](Hourly),HeatingGasEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),LightingEnergy[J](Hourly)
0,cle_com_elec_10,01/01 01:00:00,E,Exist_LowE,1.533456e+06,0,6.662803e+05,0.0,11869.19907
1,cle_com_elec_10,01/01 02:00:00,E,Exist_LowE,1.787081e+06,0,8.883738e+05,0.0,11869.19907
2,cle_com_elec_10,01/01 03:00:00,E,Exist_LowE,1.437250e+06,0,6.662803e+05,0.0,11869.19907
3,cle_com_elec_10,01/01 04:00:00,E,Exist_LowE,1.860285e+06,0,8.883738e+05,0.0,11869.19907
4,cle_com_elec_10,01/01 05:00:00,E,Exist_LowE,1.584762e+06,0,6.662803e+05,0.0,11869.19907
...,...,...,...,...,...,...,...,...,...
105115,cle_com_elec_09,12/31 20:00:00,N,Exist_LowE,0.000000e+00,0,0.000000e+00,0.0,11869.19907
105116,cle_com_elec_09,12/31 21:00:00,N,Exist_LowE,1.276364e+06,0,1.406848e+06,0.0,11869.19907
105117,cle_com_elec_09,12/31 22:00:00,N,Exist_LowE,5.762689e+05,0,1.406848e+06,0.0,11869.19907
105118,cle_com_elec_09,12/31 23:00:00,N,Exist_LowE,7.915104e+05,0,7.034239e+05,0.0,11869.19907


In [41]:
# Add energy totals
# divide by the number of seconds in an hour (3600)
totalComPeak = (get_peak_ComElec["FanEnergy[J](Hourly)"] + get_peak_ComElec["HeatingElecEnergy[J](Hourly)"] + get_peak_ComElec["HeatingGasEnergy[J](Hourly)"] + get_peak_ComElec["CoolingEnergy[J](Hourly)"] + get_peak_ComElec["LightingEnergy[J](Hourly)"])
        
get_peak_ComElec["PeakEnergy[J](Hourly)"] = totalComPeak

get_peak_ComElec["PeakEnergy(w)"] = (get_peak_ComElec["PeakEnergy[J](Hourly)"] / 3600)

get_peak_ComElec["PeakEnergy(w/sf)"] = (get_peak_ComElec["PeakEnergy(w)"] / sf)

get_peak_ComElec

,Scenario,Date_Time,Elevation,GlazingSystem,HeatingElecEnergy[J](Hourly),HeatingGasEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),LightingEnergy[J](Hourly),PeakEnergy[J](Hourly),PeakEnergy(w),PeakEnergy(w/sf)
0,cle_com_elec_10,01/01 01:00:00,E,Exist_LowE,1.533456e+06,0,6.662803e+05,0.0,11869.19907,2.211605e+06,614.334807,1.864163
1,cle_com_elec_10,01/01 02:00:00,E,Exist_LowE,1.787081e+06,0,8.883738e+05,0.0,11869.19907,2.687324e+06,746.478930,2.265146
2,cle_com_elec_10,01/01 03:00:00,E,Exist_LowE,1.437250e+06,0,6.662803e+05,0.0,11869.19907,2.115400e+06,587.611011,1.783071
3,cle_com_elec_10,01/01 04:00:00,E,Exist_LowE,1.860285e+06,0,8.883738e+05,0.0,11869.19907,2.760528e+06,766.813385,2.326850
4,cle_com_elec_10,01/01 05:00:00,E,Exist_LowE,1.584762e+06,0,6.662803e+05,0.0,11869.19907,2.262911e+06,628.586456,1.907408
...,...,...,...,...,...,...,...,...,...,...,...,...
105115,cle_com_elec_09,12/31 20:00:00,N,Exist_LowE,0.000000e+00,0,0.000000e+00,0.0,11869.19907,1.186920e+04,3.297000,0.010005
105116,cle_com_elec_09,12/31 21:00:00,N,Exist_LowE,1.276364e+06,0,1.406848e+06,0.0,11869.19907,2.695081e+06,748.633508,2.271684
105117,cle_com_elec_09,12/31 22:00:00,N,Exist_LowE,5.762689e+05,0,1.406848e+06,0.0,11869.19907,1.994986e+06,554.162788,1.681574
105118,cle_com_elec_09,12/31 23:00:00,N,Exist_LowE,7.915104e+05,0,7.034239e+05,0.0,11869.19907,1.506804e+06,418.556550,1.270085


In [42]:
# Get scenario and find the max
scenario1 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_01"]
scenario2 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_02"]
scenario3 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_03"]
scenario4 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_04"]
scenario5 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_05"]
scenario6 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_06"]
scenario7 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_07"]
scenario8 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_08"]
scenario9 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_09"]
scenario10 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_10"]
scenario11 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_11"]
scenario12 = get_peak_ComElec[get_peak_ComElec["Scenario"] == "cle_com_elec_12"]

In [43]:
scenarioPeak1 = (scenario1[scenario1["PeakEnergy[J](Hourly)"] == scenario1["PeakEnergy[J](Hourly)"].max()])
scenarioPeak2 = (scenario2[scenario2["PeakEnergy[J](Hourly)"] == scenario2["PeakEnergy[J](Hourly)"].max()])
scenarioPeak3 = (scenario3[scenario3["PeakEnergy[J](Hourly)"] == scenario3["PeakEnergy[J](Hourly)"].max()])
scenarioPeak4 = (scenario4[scenario4["PeakEnergy[J](Hourly)"] == scenario4["PeakEnergy[J](Hourly)"].max()])
scenarioPeak5 = (scenario5[scenario5["PeakEnergy[J](Hourly)"] == scenario5["PeakEnergy[J](Hourly)"].max()])
scenarioPeak6 = (scenario6[scenario6["PeakEnergy[J](Hourly)"] == scenario6["PeakEnergy[J](Hourly)"].max()])
scenarioPeak7 = (scenario7[scenario7["PeakEnergy[J](Hourly)"] == scenario7["PeakEnergy[J](Hourly)"].max()])
scenarioPeak8 = (scenario8[scenario8["PeakEnergy[J](Hourly)"] == scenario8["PeakEnergy[J](Hourly)"].max()])
scenarioPeak9 = (scenario9[scenario9["PeakEnergy[J](Hourly)"] == scenario9["PeakEnergy[J](Hourly)"].max()])
scenarioPeak10 = (scenario10[scenario10["PeakEnergy[J](Hourly)"] == scenario10["PeakEnergy[J](Hourly)"].max()])
scenarioPeak11 = (scenario11[scenario11["PeakEnergy[J](Hourly)"] == scenario11["PeakEnergy[J](Hourly)"].max()])
scenarioPeak12 = (scenario12[scenario12["PeakEnergy[J](Hourly)"] == scenario12["PeakEnergy[J](Hourly)"].max()])

In [44]:
PeakDemandElec =scenarioPeak1.append([scenarioPeak2, 
                                  scenarioPeak3,
                                  scenarioPeak4,
                                  scenarioPeak5,
                                  scenarioPeak6,
                                  scenarioPeak7,
                                  scenarioPeak8,
                                  scenarioPeak9,
                                  scenarioPeak10,
                                  scenarioPeak11,
                                  scenarioPeak12])

PeakDemandElec

,Scenario,Date_Time,Elevation,GlazingSystem,HeatingElecEnergy[J](Hourly),HeatingGasEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),LightingEnergy[J](Hourly),PeakEnergy[J](Hourly),PeakEnergy(w),PeakEnergy(w/sf)
27151,cle_com_elec_01,02/06 08:00:00,N,Exist,20691710.05,0,1959090.598,0.0,71215.1944,2.272202e+07,6311.671067,19.152393
35911,cle_com_elec_02,02/06 08:00:00,E,Exist,21260780.14,0,1133606.621,0.0,71215.1944,2.246560e+07,6240.444988,18.936262
44671,cle_com_elec_03,02/06 08:00:00,S,Exist,20204067.82,0,1835447.945,0.0,71215.1944,2.211073e+07,6141.869711,18.637141
53431,cle_com_elec_04,02/06 08:00:00,W,Exist,20701433.11,0,1644337.018,0.0,71215.1944,2.241699e+07,6226.940367,18.895283
62191,cle_com_elec_05,02/06 08:00:00,N,Exist_Clear,15997547.11,0,1773405.401,0.0,71215.1944,1.784217e+07,4956.157696,15.039168
70951,cle_com_elec_06,02/06 08:00:00,E,Exist_Clear,16612500.02,0,1105630.595,0.0,71215.1944,1.778935e+07,4941.484947,14.994644
79711,cle_com_elec_07,02/06 08:00:00,S,Exist_Clear,15175402.95,0,1664468.559,0.0,71215.1944,1.691109e+07,4697.524084,14.254359
88471,cle_com_elec_08,02/06 08:00:00,W,Exist_Clear,16225577.92,0,1465456.945,0.0,71215.1944,1.776225e+07,4933.958350,14.971805
97231,cle_com_elec_09,02/06 08:00:00,N,Exist_LowE,16624544.58,0,1406847.888,0.0,71215.1944,1.810261e+07,5028.502128,15.258693
871,cle_com_elec_10,02/06 08:00:00,E,Exist_LowE,17063908.53,0,888373.791,0.0,71215.1944,1.802350e+07,5006.527088,15.192011


In [45]:
# Export dataframe  to csv
PeakDemandElec.to_csv("Scraper_Output/cle_peak_com.csv", header=True, index=True)